# Check for confounding variables

This notebook uses chi-squared tests to look for clinical variables that are associated with having a chromosome event or not.

- Get clinical tables
- Get event tables
- Binarize clinical columns as needed
- For each binary column in the clinical table, make a contingency table of that column and the event table
- Run chi squared test and save results

In [ ]:
import pandas as pd
import numpy as np
import os
import cptac

In [ ]:
CANCER_TYPES = [
    "brca",
    "colon",
    "hnscc",
    "lscc",
    "luad",
    "ovarian"
]

In [ ]:
def find_confounding_vars_cancer_type(cancer_type):
    pass